# 2020-01-21 UT Preliminary Analysis

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
from imaka.analysis import plot_stats
from astropy.table import Table
import numpy as np
from importlib import reload
from imaka.reduce.nights import reduce_2020_01_21 as redu

reload(redu)
reload(plot_stats)

root_dir = "/g/lu/data/imaka/onaga/20200121/sta/"
print("Root directory:",root_dir)
stats_dir = root_dir + "reduce/stats/"
plots_dir = root_dir + "reduce/plots/"
stack_dir = root_dir + "reduce/stacks/"

Root directory: /g/lu/data/imaka/onaga/20200121/sta/


# Data Reduction

Fully reduce the data from 20200121 UT. Don't re-run this unless you really need to as it takes awhile. After you re-run the data reduction, please remember to load it back up to onaga.

In [8]:
"""Set the appropriate root directory for your machine"""
redu.root_dir = root_dir

""""Make sky and flat images and use them to reduce science images"""
#redu.make_sky() #using sky img from 2020-01-22 run (60s exposure time)
#redu.make_flat()  # Using twilight images from 20200122 run
#redu.reduce_FLD2()

""""Find stars in clean images and calculate stats"""
# redu.find_stars_FLD2()
# redu.calc_star_stats()

""""Split four filter data and calculate stats"""
redu.split_filt()
redu.calc_star_stats_fourfilt()

""""Make a stack of images for each mode and analyze stacks"""
redu.stack_FLD2()
redu.analyze_stacks()

Splitting file 1 of 5
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta065_o_scan_clean_stars.txt
Splitting file 2 of 5
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta067_o_scan_clean_stars.txt
Splitting file 3 of 5
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta070_o_scan_clean_stars.txt
Splitting file 4 of 5
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta079_o_scan_clean_stars.txt
Splitting file 5 of 5
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta081_o_scan_clean_stars.txt
Splitting file 1 of 6
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta083_o_scan_clean_stars.txt
Splitting file 2 of 6
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta085_o_scan_clean_stars.txt
Splitting file 3 of 6
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta087_o_scan_clean_stars.txt
Splitting file 4 of 6
	 /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta089_o_scan_clean_stars.txt
Splitting file 5 of 6
	 /g/lu/data/im

Working on image:  /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta067_o_scan_clean.fits
Working on image:  /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta070_o_scan_clean.fits
Working on image:  /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta079_o_scan_clean.fits
Working on image:  /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta081_o_scan_clean.fits
Working on image:  /g/lu/data/imaka/onaga/20200121/sta/reduce/Beehive-W/sta083_o_scan_clean.fits


/u/caylananderson/code/python/imaka/imaka/reduce/reduce_STA.py:517: RuntimeWarning: invalid value encountered in greater_equal
  ee25_rad = radii[ np.where(enc_energy_final >= 0.25)[0][0]]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import pdb
pdb.pm()

> /u/caylananderson/code/python/imaka/imaka/reduce/reduce_STA.py(517)calc_star_stats()
-> ee25_rad = radii[ np.where(enc_energy_final >= 0.25)[0][0]]
(Pdb) print(enc_energy_final)
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan]
(Pdb) type(enc_energy_final)
<class 'numpy.ndarray'>
(Pdb) print(bkg_annuli)
Aperture: CircularAnnulus
positions: [[3030.158, 4544.678],
            [4939.015, 5102.371]]
r_in: 93.75
r_out: 125.0
(Pdb) print(bkg)
 id xcenter  ycenter      aperture_sum   
      pix      pix                       
--- -------- -------- -------------------
  1 3030.158 4544.678 -262814.13030611584
  2 4939.015 5102.371                 nan
(Pdb) print(img)
[[ -39.90704398   37.06788716 -103.48672972 ...   38.2862644
    95.94729658  135.77917932]
 [-159.88462324  110.82671756  -68.42772489 ...

### 